In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

     

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

f=open('/content/drive/My Drive/ferdosi.txt',encoding='utf-8')
first_part=[]
second_part=[]
max_first_part=0
max_second_part=0
chars=set()
for lines in f:
    lines=lines.strip()
    x,y=lines.split(',')
    input_texts.append(x)
    target_texts.append(y)
    max_first_part=max(max_first_part,len(x))
    max_second_part=max(max_second_part,len(y))
    for ch in lines:
        input_characters.add(ch)
f.close() 
input_characters=list(input_characters)
input_characters.remove(',') 
PAD='_PAD_'
BOM='_BOM_'
EOM='_EOM_'
input_characters=[BOM,PAD,EOM]+input_characters 
target_characters=input_characters




In [ ]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length= max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#for use _EOM_ & _BOM_
max_encoder_seq_length+=2 
max_decoder_seq_length+=2

#make our dict
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])



encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    encoder_input_data[i, :max_encoder_seq_length-len(input_text)-1, input_token_index['_PAD_']] = 1.
    encoder_input_data[i, max_encoder_seq_length-len(input_text)-2, input_token_index['_BOM_']] = 1.
    for t, char in enumerate(input_text):
        encoder_input_data[i, max_encoder_seq_length-len(input_text)-1+t, input_token_index[char]] = 1.
    encoder_input_data[i, max_encoder_seq_length-1, input_token_index['_EOM_']] = 1.
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, input_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t-1 , input_token_index[char]] = 1.
    decoder_input_data[i, t+1, input_token_index['_EOM_']] = 1.
    decoder_input_data[i, t+2:, input_token_index['_PAD_']] = 1.
    decoder_target_data[i, t, input_token_index['_EOM_']] = 1.
    decoder_target_data[i, t+1:, input_token_index['_PAD_']] = 1.

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')



Train on 39687 samples, validate on 9922 samples
Epoch 1/100
39687/39687 [==============================] - 75s 2ms/step - loss: 1.6731 - acc: 0.5191 - val_loss: 1.4510 - val_acc: 0.5679
Epoch 2/100
39687/39687 [==============================] - 71s 2ms/step - loss: 1.3121 - acc: 0.6104 - val_loss: 1.2933 - val_acc: 0.6161
Epoch 3/100
39687/39687 [==============================] - 71s 2ms/step - loss: 1.1916 - acc: 0.6453 - val_loss: 1.2103 - val_acc: 0.6386
Epoch 4/100
39687/39687 [==============================] - 70s 2ms/step - loss: 1.1171 - acc: 0.6671 - val_loss: 1.1493 - val_acc: 0.6619
Epoch 5/100
39687/39687 [==============================] - 70s 2ms/step - loss: 1.0644 - acc: 0.6817 - val_loss: 1.1141 - val_acc: 0.6717
Epoch 6/100
39687/39687 [==============================] - 69s 2ms/step - loss: 1.0256 - acc: 0.6927 - val_loss: 1.0916 - val_acc: 0.6769
Epoch 7/100
39687/39687 [==============================] - 69s 2ms/step - loss: 0.9948 - acc: 0.7013 - val_loss: 1.0655 - v

In [ ]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['_BOM_']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_input_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_EOM_' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: به نام خداوند جان و خرد 
Decoded sentence: نماند ان زمان روزگاران مرد_EOM_
-
Input sentence: خداوند نام و خداوند جای 
Decoded sentence: همان نامداری و فرخ سپای_EOM_
-
Input sentence: خداوند کیوان و گردان سپهر 
Decoded sentence: بید انچش دیو ازاده ست_EOM_
-
Input sentence: ز نام و نشان و گمان برترست 
Decoded sentence: و گر بر نهان کار او راه بست_EOM_
-
Input sentence: به بینندگان افریننده را 
Decoded sentence: بد از تن سر از تن به یک روز را_EOM_
-
Input sentence: نیابد بدو نیز اندیشه راه 
Decoded sentence: نه سر بر سر افگند بر کار راه_EOM_
-
Input sentence: سخن هر چه زین گوهران بگذرد 
Decoded sentence: ندید از برش روزگاران کرد_EOM_
-
Input sentence: خرد گر سخن برگزیند همی 
Decoded sentence: منان دان به اندیشه از امدش_EOM_
-
Input sentence: ستودن نداند کس او را چو هست 
Decoded sentence: سخن گشت با درد و سنگ امدست_EOM_
-
Input sentence: خرد را و جان را همی سنجد اوی 
Decoded sentence: بران نامور شهریار از جوی_EOM_
